# Examples 

As our main examples we will consider Hilbert modular groups of the following three number Fields:
- $ K_1 = \mathbb{Q}(\sqrt{5})$  (one cusp, discriminant 5, generally the simplest example)
- $ K_2 = \mathbb{Q}(\sqrt{10})$ (two cusps, discriminant 40)
- $ K_3 = \mathbb{Q}(\alpha)$, where $\alpha$ has minimal polynomial $\alpha^3-\alpha^2-2x+1$ (one cusp, discriminant 49)

In this notebook we will look at the second example. 

## Example 2  $K_2=\mathbf{Q}(\sqrt{10})$

This has class number 2, discriminant $40$ and fundamental unit $\epsilon=3+\sqrt{10}$.

Note that SageMath orders the embeddings as $-\sqrt{10},+\sqrt{10}$ chooses the fundamental unit $3-\sqrt{10}$.

## Introductory example of basic usage
Let's first demonstrate the reduction of a point $$z=(0.1+i,0.2+0.5i) \in \mathbb{H}^2$$ with respect to the fundamental domain of $K_2$.

In [ ]:
from plot import plot_polygon
from hilbert_modgroup.all import HilbertModularGroup, HilbertPullback, UpperHalfPlaneProductElement
H2 = HilbertModularGroup(10)
K2 = H2.base_ring().number_field()
P2 = HilbertPullback(H2)

In [ ]:
z = UpperHalfPlaneProductElement([0.1+I,0.2+0.5*I])
P2.reduce(z)

Then check that if we apply a random element of $H_2$ we get the same (up to numerical error) reduced point

In [ ]:
w = H2.random_element(-5,5).acton(z)
P2.reduce(z) - P2.reduce(w)

## Details of the algorithm
We will now go through the inner workings of the algorithm and the construction of the relevant bounds in more detail.
Note that the functions below will mainly be of interest to researchers who wants to verify or extend these algorithms or results.

The two cusps of $\mathrm{SL}(\mathcal{O}_{K_2})$ can be represented by 
$\lambda_1=\infty=(1:0)$ and $\lambda_2=(2:\sqrt{10})$ corresponding to the representative ideals
$\mathfrak{a}_1=\mathcal{O}_{K_2}$ and $\mathfrak{a}_2=(2,\sqrt{10})$ where $\mathbb(\mathfrak{a}_2)=2$.

It is worth noting that the class group representative given by Sage is $(3,2+\sqrt{10})$ of norm $3$.

In [ ]:
P2._matrix_BLambda_row_sum(1)

In [ ]:
H2.cusps()

In [ ]:
c=H2.cusps()[1]
(c.numerator()/c.denominator()).complex_embeddings()

The ring of integers has basis $\alpha_1=1$ and $\alpha_2=\sqrt{10}$ so we have 

$$
B_{\mathcal{O}_{K_2}} = 
\left(\begin{array}{cc}
1 & \sqrt{10}\\
1 & -\sqrt{10}
\end{array}\right)
\quad
B_{\mathcal{O}_{K_2}}^{-1}=\frac{1}{-2\sqrt{10}}\left(\begin{array}{cc}
-\sqrt{10} & -\sqrt{10}\\
-1 & 1
\end{array}\right)
$$
and
$$B_{\Lambda} = \left(\log (3+\sqrt{10})  \log (\sqrt{10}-3)\right).$$

Hence $$\left\Vert B_{\Lambda}\right\Vert _{\infty}	\approx1.81, \quad D_{0}\approx4.30,$$
$$\left\Vert B_{\mathcal{O}_K}\right\Vert _{\infty}\approx4.16 \quad  \left\Vert B_{\mathcal{O}_K}^{-1}\right\Vert _{\infty}=1.$$

In [ ]:
P2.D()

In [ ]:
P2.basis_matrix_ideal().norm(Infinity)

In [ ]:
P2.basis_matrix_ideal().inverse().norm(Infinity)

In [ ]:
P2.basis_matrix_logarithmic_unit_lattice().norm(Infinity)

Let's find the closest cusp to the point $\mathbf{z}=i\mathbf{1}$.

In [ ]:
z=UpperHalfPlaneProductElement([CC(0,1.0),CC(0,1.0)])

First of all we note that the preliminary LLL method does not yield any other cusps than infinity to compare with.


In [ ]:
P2.get_heuristic_closest_cusp(z)

The norm bound is now: 

In [ ]:
P2.max_ideal_norm()

In [ ]:
# Add arguments to include the norm bounds
norm_args = {'norm_bound':P2._bound_for_sigma_norm(z)}

In [ ]:
p=P2._candidate_integers_sigma(z,domain='preimage',return_polyhedron=True)
plot_polygon(p,K2.fractional_ideal(1).integral_basis(),action='show',xmin=-5,xmax=5,ymin=-4.5,ymax=4.5,**norm_args)

In [ ]:
plot_polygon(p,K2.fractional_ideal(1).integral_basis(),action='save',xmin=-5,xmax=5,ymin=-4.5,ymax=4.5,filename='K2.z1.domain1.pgf',**norm_args)

In [ ]:
p=P2._candidate_integers_sigma(z,domain='polytope',return_polyhedron=True)
print(p.vertices())
# Add arguments to include the norm bounds
norm_args = {'norm_bound':P2._bound_for_sigma_norm(z),'curve_map':P2.basis_matrix_ideal().inverse(),'norm_plot_factor':1.5}
plot_polygon(p,[1,1],action='show',ticks=[2,2],xmin=-4.75,xmax=4.75,ymin=-4.75,ymax=4.75,**norm_args)

In [ ]:
plot_polygon(p,[1,1],action='save',ticks=[2,2],filename='K2.z1.domain2.pgf',xmin=-4.75,xmax=4.75,ymin=-4.75,ymax=4.75,**norm_args)

We can see now that the norm bounds are actually very efficient in this case as they eliminate 6 potential $\sigma$s and leave only $0$ ,$1$ and $-1$.

In [ ]:
P2._candidate_integers_sigma(z)

We obtain 10 candidates for closest cusp and we see that the cusps $0$ and $\infty$ are both closest with distance $1$.

In [ ]:
candidate_cusps = P2._candidate_closest_cusps(z,as_cusps=True)
print("Number of candidate cusps=",len(candidate_cusps))
for c in candidate_cusps:
    print(P2.distance_to_cusp(c,z),c)

In [ ]:
P2.find_closest_cusp(z,return_multiple=True)

##  Another point
Let's consider moving the point further towards 0 and consider again $\mathbf{z}=\frac{1}{2}i\mathbf{1}$.

In [ ]:
z=UpperHalfPlaneProductElement([CC(0,0.5),CC(0.0,0.5)])

In this case the preliminary search yields the cusp $(0:1)$ with distance $1/2$ so we will use the algorithm with $d=1/2$.

In [ ]:
c=P2.get_heuristic_closest_cusp(z); c

In [ ]:
P2.distance_to_cusp(c,z)

In [ ]:
P2._bound_for_sigma_norm(z,dist=0.5)

So the norm bound is the same again as before.

In [ ]:
p=P2._candidate_integers_sigma(z,domain='preimage',return_polyhedron=True)
norm_args = {'norm_bound':P2._bound_for_sigma_norm(z,dist=0.5),'xmin':-35,'xmax':35}
plot_polygon(p,K2.fractional_ideal(1).integral_basis(),action='show',**norm_args)

In [ ]:
plot_polygon(p,K2.fractional_ideal(1).integral_basis(),action='save',filename='K2.z2.domain1.png',**norm_args)

In [ ]:
p=P2._candidate_integers_sigma(z,domain='polytope',return_polyhedron=True)
norm_args = {'norm_bound':P2._bound_for_sigma_norm(z,dist=0.5),'curve_map':P2.basis_matrix_ideal().inverse(),
             'norm_plot_factor':1.5,'ymin':-7,'ymax':7}
plot_polygon(p,[1,1],action='show',**norm_args)

In [ ]:
plot_polygon(p,[1,1],action='save',filename='K2.z2.domain2.png',**norm_args)

It is clear from these images that we again have only $\sigma = 0, -1, 1$ and the potential cusps are also precisely $0$, $1$ and $-1$

In [ ]:
P2._candidate_integers_sigma(z)

In [ ]:
candidate_cusps = P2._candidate_closest_cusps(z,as_cusps=True)
print("Number of candidate cusps=",len(candidate_cusps))
for c in candidate_cusps:
    print(P2.distance_to_cusp(c,z),c)

And it is clear that $0$ is the unique closest cusp. 

In [ ]:
P2.find_closest_cusp(z,return_multiple=True)

Now, if we didn't use the preliminary reduction we would have gotten $13$ possible values for $\sigma$ and 47 candidate cusps. 

In [ ]:
len(P2._candidate_integers_sigma(z,use_initial_bd_d=False))

In [ ]:
len(P2._candidate_closest_cusps(z,use_initial_bd_d=False))

## A point close to the second cusp
So far we have only involved cusps equivalent to infinity. 
To demonstrate the the algorithm works for other cusps, consider the following point 


In [ ]:
z=UpperHalfPlaneProductElement([CC(2.58,0.5),CC(0.5,0.5)])

In [ ]:
c=P2.get_heuristic_closest_cusp(z); c

In [ ]:
P2.distance_to_cusp(c,z)

In [ ]:
P2._bound_for_sigma_norm(z,dist=1.59731464318420)

In [ ]:
p=P2._candidate_integers_sigma(z,domain='preimage',return_polyhedron=True)
norm_args = {'norm_bound':P2._bound_for_sigma_norm(z,dist=1.59731464318420)}
plot_polygon(p,K2.fractional_ideal(1).integral_basis(),action='show',ticks=[5,5],**norm_args)

In [ ]:
plot_polygon(p,K2.fractional_ideal(1).integral_basis(),action='save',ticks=[5,5],filename='K2.z3.domain1.pgf',**norm_args)

In [ ]:
p=P2._candidate_integers_sigma(z,domain='polytope',return_polyhedron=True)
norm_args = {'norm_bound':P2._bound_for_sigma_norm(z,1.59731464318420),'curve_map':P2.basis_matrix_ideal().inverse(),'norm_plot_factor':1.5}
plot_polygon(p,[1,1],action='show',ymax=5,ymin=-5,xmax=5,xmin=-5,**norm_args)

In [ ]:
plot_polygon(p,[1,1],action='save',ticks=[4,4],filename='K2.z3.domain2.pgf',ymax=5,ymin=-5,xmax=5,xmin=-5,**norm_args)

And we see again that the norm bound actually does remove a lot of potential $\sigma$s.

In [ ]:
sigmas = P2._candidate_integers_sigma(z)
print(len(sigmas))

In [ ]:
candidate_cusps = P2._candidate_closest_cusps(z,as_cusps=True)
print("Number of candidate cusps=",len(candidate_cusps))

So by using these bounds we only need to compare the distance to 28 distinct cusps. 

If we had not used the norm bound we would have had to consider 89 cusps:

In [ ]:
len(P2._candidate_closest_cusps(z,use_norm_bound=False,use_initial_bd_d=False))

In [ ]:
c = P2.find_closest_cusp(z,return_multiple=True)[0]
c

In [ ]:
P2.distance_to_cusp(c,z)

This cusp is equivalent to the second cusp: 

In [ ]:
c.is_Gamma0_equivalent(H2.cusps()[0],K2.fractional_ideal(1)) # Not equivalent to infinity

In [ ]:
c.is_Gamma0_equivalent(H2.cusps()[1],K2.fractional_ideal(1),True) #Is equivalent to the other cusp.

In [ ]:
l=_[1]

In [ ]:
c.apply(l)

In [ ]:
P2.distance_to_cusp(c,z)

Now, let's see the complete pull-back to the fundamental domain:

In [ ]:
z=UpperHalfPlaneProductElement([CC(2.58,0.5),CC(0.5,0.5)])
w, B = P2.reduce(z,return_map=True)
w,B

Let's check that this point is correct:

In [ ]:
# Check that the map takes z to w
z.apply(B) - w

In [ ]:
r,s=P2.get_heuristic_closest_cusp(w)
c = NFCusp(K2,r,s)
print("Preliminary closest cusp (by LLL)=",c)
c == H2.cusps()[1]

So we see that this is actually another representative of the same cusp as before. 
However, we need of course to **prove** that this is the closest cusp.


In [ ]:
# This might take a few minutes (took 130s minutes on my laptop). Uncomment if you want to make sure, 
# otherwise use the cusps in the next cell
# import time
# t0 = time.time()
# l=P2._candidate_closest_cusps(w)
# t1 = time.time()
# print(t1-t0)
# 130.6130931377411

In [ ]:
a = P2.number_field().gen()
l=[(1, 0), (-7*a - 22, -10*a - 31), (-7*a - 20, -9*a - 31), (-6*a - 18, -8*a - 27), (-6*a - 16, -7*a - 27),
     (-5*a - 13, -6*a - 22), (-5*a - 11, -5*a - 22), (-4*a - 16, -7*a - 18), (-4*a - 9, -4*a - 18),
     (-3*a - 7, -3*a - 14), (-3*a - 5, -2*a - 14), (-3*a + 2, a - 14), (-3*a + 4, 2*a - 14), (-3*a - 13, -6*a - 13),
     (-3*a - 11, -5*a - 13), (-3*a, -13), (-2*a + 6, 3*a - 10), (-2*a - 11, -5*a - 9), (-2*a - 9, -4*a - 9), 
     (-2*a, -9), (-2*a + 2, a - 9), (-2*a + 9, 4*a - 9), (-a - 7, -3*a - 5), (-a + 11, 5*a - 5), (-a - 4, -2*a - 4), 
     (-a - 2, -a - 4), (-11, -5*a), (-9, -4*a), (-7, -3*a)]
len(l) 

In [ ]:
for r,s in l:
    d = P2.distance_to_cusp(NFCusp(K2,r,s),w)
    print(f"{str(r):<10} {str(s):<15}{str(d)}")

And we see that the closest cusp is given by (-2*a + 6:3*a - 10) which is just another representative of the cusp $\lambda_2$ so $w$ is indeed closest to this cusp:

In [ ]:
NFCusp(K2,-2*a + 6,3*a - 10) == H2.cusps()[1]

We can now also check that it is reduced with respect to the stabiliser of $\lambda_2$:

In [ ]:
P2.X(w,H2.ideal_cusp_representatives()[1]**(-2))

In [ ]:
P2.Y(w)

And hence the point w is indeed in the fundamental domain.